# TOPIC PREDICTION

### Import Library

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import gensim

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.model_selection import train_test_split
import pickle as pkl # Media nyimpan ...
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim import corpora, models

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Data Exploration

In [ ]:
df3 = pd.read_csv('/content/data_3A.csv')

In [ ]:
df3.head(5)

,Unnamed: 0,ABSTRACT
0,0,Predictive models allow subject-specific inf...
1,1,Rotation invariance and translation invarian...
2,2,We introduce and develop the notion of spher...
3,3,The stochastic Landau--Lifshitz--Gilbert (LL...
4,4,Fourier-transform infra-red (FTIR) spectra o...


In [ ]:
df3.tail(5)

,Unnamed: 0,ABSTRACT
95,95,"In this paper, we presented a novel convolut..."
96,96,A variety of representation learning approac...
97,97,Motivated by Perelman's Pseudo Locality Theo...
98,98,We bound an exponential sum that appears in ...
99,99,We investigate the effect of dimensional cro...


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   ABSTRACT    100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
df3.drop(columns=['Unnamed: 0'], inplace=True)
df3.head(5)

,ABSTRACT
0,Predictive models allow subject-specific inf...
1,Rotation invariance and translation invarian...
2,We introduce and develop the notion of spher...
3,The stochastic Landau--Lifshitz--Gilbert (LL...
4,Fourier-transform infra-red (FTIR) spectra o...


### Data Cleaning / Data Preparation

Setelah menjelajahi dan mengenal data, proses yang dilakukan selanjutnya yaitu membersihkan data, serta mengubahnya ke dalam bentuk token, yang akan mempermudah proses analisa selanjutnya.

In [ ]:
def cleaning(df):
    # Ubah ke Lowercase
    clean_text = df.str.casefold()
    # Menghapus Angka
    clean_text = [re.sub(r'\d+', '', i ) for i in clean_text]
    # Menghapus tanda baca
    clean_text = [re.sub(r'[^\w]', ' ', i) for i in clean_text]
    # Menghapus Spasi Double
    clean_text = [re.sub(r'\s+', ' ', i) for i in clean_text]
    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    clean_text = [' '.join([word for word in word_tokenize(text) if word.lower() not in stop_words]) for text in clean_text]
    # Tokenisasi
    clean_text = [word_tokenize(text) for text in clean_text]
    # Lemmatisasi
    lemmatizer = WordNetLemmatizer()
    clean_text = [[lemmatizer.lemmatize(word) for word in text] for text in clean_text]

    return clean_text

Variabel yang dilakukan proses Cleaning yaitu variabel "ABSTRACT". Dapat dilihat bahwa data yang saya miliki itu sudah bersih dan sudah berbentuk token, jadi setiap array akan memiliki tokennya masing - masing.

In [ ]:
proces_data = cleaning(df3['ABSTRACT'])
proces_data

[['predictive',
  'model',
  'allow',
  'subject',
  'specific',
  'inference',
  'analyzing',
  'disease',
  'related',
  'alteration',
  'neuroimaging',
  'data',
  'given',
  'subject',
  'data',
  'inference',
  'made',
  'two',
  'level',
  'global',
  'e',
  'identifiying',
  'condition',
  'presence',
  'subject',
  'local',
  'e',
  'detecting',
  'condition',
  'effect',
  'individual',
  'measurement',
  'extracted',
  'subject',
  'data',
  'global',
  'inference',
  'widely',
  'used',
  'local',
  'inference',
  'used',
  'form',
  'subject',
  'specific',
  'effect',
  'map',
  'rarely',
  'used',
  'existing',
  'model',
  'often',
  'yield',
  'noisy',
  'detection',
  'composed',
  'dispersed',
  'isolated',
  'island',
  'article',
  'propose',
  'reconstruction',
  'method',
  'named',
  'rsm',
  'improve',
  'subject',
  'specific',
  'detection',
  'predictive',
  'modeling',
  'approach',
  'particular',
  'binary',
  'classifier',
  'rsm',
  'specifically',
  'ai

BUILD DICTIONARY

Disini saya membangun sebuah function untuk membangun kamus (Dictionary) dari token yang ada di proces_data (dengan menggunakan function yang berasal dari gensim).

Setelah itu saya melakukan filterisasi kamus dengan menghapus kata - kata yang muncul yang kurang dari 10 kali (no_below=10), kata - kata yang muncul di atas 10% dari seluruh dokumen (no_above=0.1) dan hanya menyimpan 100000 kata paling umum

In [ ]:
# Bag of Words on the dataset
def build_dic(text):
    dictionary = gensim.corpora.Dictionary(text)
    dictionary.filter_extremes(no_below=10, no_above=0.1, keep_n= 100000)
    return dictionary

dictionary=build_dic(proces_data)

Kode di bawah ini berguna untuk mengonversi setiap dokumen dalam proces_data menjadi representasi Bag of Words (BOW) menggunakan kamus dictinary yang telah dibuat sebelumnya.

Kemudian saya melakukan akses representasi BoW untuk dokumen ke-80 dalam bow_corpus

In [ ]:
bow_corpus=[dictionary.doc2bow(doc) for doc in proces_data]
bow_corpus[80]

[(7, 1), (8, 1), (10, 1), (15, 1)]

Kode di bawah ini untuk mencetak informasi tentang setiap kata dalam representasi BoW untuk dokumen ke-80. Ini akan menghitung frekuensi kemunculan kata tersebut dalam dokumen ke-80.

In [ ]:
bow_corpus_80=bow_corpus[80]
for i in range(len(bow_corpus_80)):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus_80[i][0],
                                                     dictionary[bow_corpus_80[i][0]],
                                                     bow_corpus_80[i][1]))

Word 7 ("distribution") appears 1 time.
Word 8 ("observation") appears 1 time.
Word 10 ("long") appears 1 time.
Word 15 ("shown") appears 1 time.


Jadi dapat dilihat bahwa hanya terdapat 1 kata "distribution" di bow_corpus ke-80 dan seterusnya.

Disini saya membangun model TF - IDF dari 'bow_corpus' menggunakan kelas 'TfidfModel' yang ada di Gensim. Model ini akan menghitung bobot TF-IDF untuk setiap kata dalam setiap dokumen.

In [ ]:
# TF - IDF on our Document Set
def vector_tfidf(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    return corpus_tfidf

corpus_tfidf=vector_tfidf(bow_corpus)
print(corpus_tfidf)

Dengan menggunakan TF - IDF, kita dapat mengukur  tingkat kepentingan kata dalam dokumen berdasarkan frekuensi kemunculan kata tersebut dalam dokumen dan dalam keseluruhan korpus. Representasi TF-IDF ini sangat berguna dalam analisis teks dan pemodelan topik, karena dapat membantu mengidentifikasi kata-kata kunci yang memiliki kontribusi yang lebih tinggi dalam membedakan dokumen-dokumen dalam set dokumen.

Dibangun sebuah model LDA yang akan digunakan untuk mengidentifikasi topik-topik yang ada dalam set dokumen. Setelah melatih model, kita dapat melakukan analisis terhadap topik-topik yang ditemukan, seperti mengekstraksi kata-kata kunci dalam setiap topik dan menganalisis distribusi topik dalam setiap dokumen.

In [ ]:
# Running LDA Using TF - IDF Corpus
def model_lda(dictionary,bow_corpus,num_topic,alpha,eta):
    lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                   num_topics = num_topic,
                                   id2word = dictionary,
                                   passes = 10,
                                   workers = 2,
                                   alpha=alpha,
                                   eta=eta)
    return lda_model

Kode di bawah menggunakan model LDA yang telah dilatih dengan representasi TF-IDF dari set dokumen untuk menampilkan topik-topik yang ditemukan beserta kata-kata kunci dalam setiap topik.

In [ ]:
lda_model=model_lda(dictionary,corpus_tfidf,5,0.7,0.7) #5 itu Jumlah topic (ini optional) eta dan alpha 0.7 (sesuatu yg bisa di tuning)

Dalam loop ini, untuk setiap topik yang ditemukan dalam model LDA, kita mencetak nomor topik dan kata-kata kunci yang mewakili topik tersebut. Informasi ini dapat membantu dalam memahami dan menganalisis topik-topik yang terbentuk dalam set dokumen.

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.075*"measure" + 0.073*"value" + 0.068*"solution" + 0.065*"finite" + 0.064*"long" + 0.063*"local" + 0.063*"general" + 0.060*"domain" + 0.060*"distribution" + 0.058*"simple"


Topic: 1 
Words: 0.075*"general" + 0.068*"measure" + 0.064*"condition" + 0.064*"long" + 0.063*"local" + 0.063*"field" + 0.062*"finite" + 0.059*"solution" + 0.059*"observation" + 0.058*"distribution"


Topic: 2 
Words: 0.072*"measure" + 0.071*"long" + 0.068*"condition" + 0.065*"provide" + 0.064*"finite" + 0.060*"main" + 0.059*"local" + 0.059*"field" + 0.058*"simple" + 0.057*"general"


Topic: 3 
Words: 0.107*"main" + 0.082*"measure" + 0.072*"linear" + 0.062*"long" + 0.061*"provide" + 0.060*"solution" + 0.060*"general" + 0.057*"simple" + 0.056*"local" + 0.056*"finite"


Topic: 4 
Words: 0.107*"domain" + 0.100*"field" + 0.067*"observation" + 0.065*"solution" + 0.063*"condition" + 0.060*"long" + 0.059*"distribution" + 0.056*"provide" + 0.052*"main" + 0.052*"value"




Dapat dilihat dari output di atas itu menunjukkan bahwa semua kata memiliki makna dan maksudnya tersendiri, jadi Topic serta model yang dihasilkan sudah bagus dan saya memakai 5 topic.

Kode di bawah mengimplementasikan fungsi score_perf() yang digunakan untuk menghitung skor koherensi (coherence score) dari model LDA. Skor koherensi digunakan untuk mengevaluasi sejauh mana topik-topik yang dihasilkan oleh model LDA koheren dan saling terkait.

Coherence Score yang saya miliki itu hanya sebesar 0.3505 ini sangat jauh dari dikatakan bagus, karena Umumnya Coherence Score yang bagus itu bernilai 1, namun ini cenderung susah karena jumlah korpus yang dimiliki banyak. semakin banyak jumlah korpus maka persebaran kata semakin banyak sehingga hal ini akan semakin menyusahkan korpus.

In [ ]:
def score_perf(lda_model,text,dictionary):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

coherence_score=score_perf(lda_model,proces_data,dictionary)
print(coherence_score)

0.35057256920039503


In [ ]:
data_cleantest = df3['ABSTRACT']

Topic1: Menyimpan topik teratas (Topic dengan skor tertinggi) untuk setiap kalimat uji.

Score1: Menyimpan skore tertinggi untuk setiap kalimat uji.

Topic 2: Meyimpan topik kedua tertinggi untuk setiap kalimat uji.

Score2: Meyimpan skor kedua tertinggi untuk setiap kalimat uji.

In [ ]:
#Top 2 The highest score of topic
test_shape=len(data_cleantest)
topic1=[]
score1=[]
topic2=[]
score2=[]
sentence=[]
for doc in range(test_shape):
    sentence.append(data_cleantest[doc])
    topic1.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[0][0])
    score1.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[0][1])
    topic2.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[1][0])
    score2.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[1][1])

In [ ]:
sentence=pd.DataFrame(sentence,columns=['sentence']).reset_index()
topic1=pd.DataFrame(topic1,columns=['topic1']).reset_index()
score1=pd.DataFrame(score1,columns=['score1']).reset_index()
topic2=pd.DataFrame(topic2,columns=['topic2']).reset_index()
score2=pd.DataFrame(score2,columns=['score2']).reset_index()
test_result=pd.concat([sentence,topic1,score1,topic2,score2],axis=1)

In [ ]:
test_result=test_result[['sentence','topic1','score1','topic2','score2']]
test_result.head(10)

,sentence,topic1,score1,topic2,score2
0,Predictive models allow subject-specific inf...,2,0.293881,1,0.265587
1,Rotation invariance and translation invarian...,0,0.231032,1,0.196025
2,We introduce and develop the notion of spher...,0,0.200000,1,0.200000
3,The stochastic Landau--Lifshitz--Gilbert (LL...,4,0.354818,0,0.232510
4,Fourier-transform infra-red (FTIR) spectra o...,0,0.200000,1,0.200000
5,Let $\Omega \subset \mathbb{R}^n$ be a bound...,4,0.478962,1,0.141065
6,We observed the newly discovered hyperbolic ...,4,0.220943,1,0.202432
7,The ability of metallic nanoparticles to sup...,4,0.365199,1,0.163164
8,We model large-scale ($\approx$2000km) impac...,0,0.216395,4,0.214815
9,Time varying susceptibility of host at indiv...,4,0.221040,0,0.211172


Output di atas merupakan Persona Analysis terhadap beberapa dataset.

Output di atas berhasil memprediksi topic untuk setiap data/sentence, dimana untuk sentence ke-0 itu menunjukkan Topic kedua memiliki score 0.293881 sedangkan topic pertama memiliki score 0.265587

Untuk menentukan dominan antara topic 1 dengan topic 2 itu bisa dilihat dari score nya, jika berbeda hingga 0.2 itu menandakan bahwa kita bisa mempercayai hasil yang keluar, kalau tidak kita bisa cek sendiri.

In [ ]:
proces_data[0]

['predictive',
 'model',
 'allow',
 'subject',
 'specific',
 'inference',
 'analyzing',
 'disease',
 'related',
 'alteration',
 'neuroimaging',
 'data',
 'given',
 'subject',
 'data',
 'inference',
 'made',
 'two',
 'level',
 'global',
 'e',
 'identifiying',
 'condition',
 'presence',
 'subject',
 'local',
 'e',
 'detecting',
 'condition',
 'effect',
 'individual',
 'measurement',
 'extracted',
 'subject',
 'data',
 'global',
 'inference',
 'widely',
 'used',
 'local',
 'inference',
 'used',
 'form',
 'subject',
 'specific',
 'effect',
 'map',
 'rarely',
 'used',
 'existing',
 'model',
 'often',
 'yield',
 'noisy',
 'detection',
 'composed',
 'dispersed',
 'isolated',
 'island',
 'article',
 'propose',
 'reconstruction',
 'method',
 'named',
 'rsm',
 'improve',
 'subject',
 'specific',
 'detection',
 'predictive',
 'modeling',
 'approach',
 'particular',
 'binary',
 'classifier',
 'rsm',
 'specifically',
 'aim',
 'reduce',
 'noise',
 'due',
 'sampling',
 'error',
 'associated',
 'using

In [ ]:
for index, score in sorted(lda_model[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.2942042052745819	 
Topic: 0.072*"measure" + 0.071*"long" + 0.068*"condition" + 0.065*"provide" + 0.064*"finite" + 0.060*"main" + 0.059*"local" + 0.059*"field" + 0.058*"simple" + 0.057*"general"

Score: 0.26473647356033325	 
Topic: 0.075*"general" + 0.068*"measure" + 0.064*"condition" + 0.064*"long" + 0.063*"local" + 0.063*"field" + 0.062*"finite" + 0.059*"solution" + 0.059*"observation" + 0.058*"distribution"

Score: 0.16430488228797913	 
Topic: 0.075*"measure" + 0.073*"value" + 0.068*"solution" + 0.065*"finite" + 0.064*"long" + 0.063*"local" + 0.063*"general" + 0.060*"domain" + 0.060*"distribution" + 0.058*"simple"

Score: 0.15150362253189087	 
Topic: 0.107*"domain" + 0.100*"field" + 0.067*"observation" + 0.065*"solution" + 0.063*"condition" + 0.060*"long" + 0.059*"distribution" + 0.056*"provide" + 0.052*"main" + 0.052*"value"

Score: 0.12525080144405365	 
Topic: 0.107*"main" + 0.082*"measure" + 0.072*"linear" + 0.062*"long" + 0.061*"provide" + 0.060*"solution" + 0.060*"gene

Untuk pembuktian dapat dilihat pada output di atas. Saya menambil data index array[0] pada 'proces_data' kemudian dapat dilihat bahwa skor paling tinggi itu 0.2942 dan skor kedua yang paling tinggi yaitu 0.2647

Hal ini membuktikan bahwa apa yg ada di tabel dengan hasil output di atas menunjukkan angka yang sama.

# TEXT SUMMARIZATION

Code di bawah ini digunakan untuk membaca dataset yang ada, karena dataset memiliki ekstensi berupa .txt maka proses pembacaan dataset itu sebagai berikut

In [ ]:
# Membaca teks dari file
with open('data_4A.txt', 'r') as file:
    text = file.read()
text

'\ufeffStem cells are cells that (a) on dividing are capable of recreating themselves in at least one daughter cell and (b) have the capacity to differentiate into several linages. Stem cells play a critical and essential role in the human body not only by providing the starting material for organs and tissues but also for their continual maintenance, growth, and renewal throughout ontogeny [e.g. hematopoietic stem cells (HSCs) → erythrocytes, neural stem cells (NSCs) → neurons, etc]. As the embryo and the fetus develop, stem cells are seeded into the various tissues and organs where they remain throughout life (Figure 1). Stem cells can also exist temporarily during embryonic development and can be artificially established ex vivo from transient stages of differentiation. Embryonic stem (ES) cells of mice, for example, are isolated by culturing transiently existing inner cell mass cells of early embryos. ES cells are cells that retain the properties of primitive ectoderm cells, i.e. t

Dalam menentukan token agak berbeda dengan yang sudah Saya lakukan sebelumnya. Kali ini Saya menggunakan sent_tokenize yang berasal dari NLTK yang membagi teks menjadi kalimat - kalimat terpisah berdasarkan aturan linguistik (setiap token dipisah berdasarkan tanda baca titik (.))

In [ ]:
# Tokenisasi teks menjadi kalimat-kalimat
sentences = sent_tokenize(text)
sentences

['\ufeffStem cells are cells that (a) on dividing are capable of recreating themselves in at least one daughter cell and (b) have the capacity to differentiate into several linages.',
 'Stem cells play a critical and essential role in the human body not only by providing the starting material for organs and tissues but also for their continual maintenance, growth, and renewal throughout ontogeny [e.g.',
 'hematopoietic stem cells (HSCs) → erythrocytes, neural stem cells (NSCs) → neurons, etc].',
 'As the embryo and the fetus develop, stem cells are seeded into the various tissues and organs where they remain throughout life (Figure 1).',
 'Stem cells can also exist temporarily during embryonic development and can be artificially established ex vivo from transient stages of differentiation.',
 'Embryonic stem (ES) cells of mice, for example, are isolated by culturing transiently existing inner cell mass cells of early embryos.',
 'ES cells are cells that retain the properties of primiti

Pertama, Saya menggunakan fungsi word_tokenize dari NLTK untuk melakukan tokenisasi teks menjadi kata-kata terpisah. Saya juga mengubah semua kata menjadi huruf kecil menggunakan lower() untuk memastikan konsistensi.
Kemudian, menggunakan list comprehension, Saya memfilter kata-kata tersebut dengan menggunakan isalnum() untuk memastikan hanya kata-kata yang terdiri dari huruf dan angka yang dimasukkan.
Setelah itu, menggunakan FreqDist dari NLTK, Saya menghitung frekuensi distribusi kata-kata dari list words. FreqDist adalah objek yang digunakan untuk menghitung frekuensi kemunculan elemen dalam sebuah list.
Terakhir, Saya mencetak freq_dist untuk menampilkan hasil frekuensi distribusi kata-kata.
Hasilnya adalah dictionary yang menampilkan kata-kata sebagai kunci dan frekuensinya sebagai nilai. Dengan menggunakan print(freq_dist), Saya dapat melihat frekuensi kemunculan setiap kata dalam teks.

In [ ]:
# Menghitung frekuensi distribusi kata-kata
words = word_tokenize(text.lower())
words = [word for word in words if word.isalnum()]
print(words)

freq_dist = FreqDist(words)
print(freq_dist)

['cells', 'are', 'cells', 'that', 'a', 'on', 'dividing', 'are', 'capable', 'of', 'recreating', 'themselves', 'in', 'at', 'least', 'one', 'daughter', 'cell', 'and', 'b', 'have', 'the', 'capacity', 'to', 'differentiate', 'into', 'several', 'linages', 'stem', 'cells', 'play', 'a', 'critical', 'and', 'essential', 'role', 'in', 'the', 'human', 'body', 'not', 'only', 'by', 'providing', 'the', 'starting', 'material', 'for', 'organs', 'and', 'tissues', 'but', 'also', 'for', 'their', 'continual', 'maintenance', 'growth', 'and', 'renewal', 'throughout', 'ontogeny', 'hematopoietic', 'stem', 'cells', 'hscs', 'erythrocytes', 'neural', 'stem', 'cells', 'nscs', 'neurons', 'etc', 'as', 'the', 'embryo', 'and', 'the', 'fetus', 'develop', 'stem', 'cells', 'are', 'seeded', 'into', 'the', 'various', 'tissues', 'and', 'organs', 'where', 'they', 'remain', 'throughout', 'life', 'figure', '1', 'stem', 'cells', 'can', 'also', 'exist', 'temporarily', 'during', 'embryonic', 'development', 'and', 'can', 'be', 'art

Disini saya melakukan penghapusan stop words.

In [ ]:
# Menghapus stopwords (kata-kata umum yang tidak memiliki makna khusus)
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word not in stop_words]
print(filtered_words)

['cells', 'cells', 'dividing', 'capable', 'recreating', 'least', 'one', 'daughter', 'cell', 'b', 'capacity', 'differentiate', 'several', 'linages', 'stem', 'cells', 'play', 'critical', 'essential', 'role', 'human', 'body', 'providing', 'starting', 'material', 'organs', 'tissues', 'also', 'continual', 'maintenance', 'growth', 'renewal', 'throughout', 'ontogeny', 'hematopoietic', 'stem', 'cells', 'hscs', 'erythrocytes', 'neural', 'stem', 'cells', 'nscs', 'neurons', 'etc', 'embryo', 'fetus', 'develop', 'stem', 'cells', 'seeded', 'various', 'tissues', 'organs', 'remain', 'throughout', 'life', 'figure', '1', 'stem', 'cells', 'also', 'exist', 'temporarily', 'embryonic', 'development', 'artificially', 'established', 'ex', 'vivo', 'transient', 'stages', 'differentiation', 'embryonic', 'stem', 'es', 'cells', 'mice', 'example', 'isolated', 'culturing', 'transiently', 'existing', 'inner', 'cell', 'mass', 'cells', 'early', 'embryos', 'es', 'cells', 'cells', 'retain', 'properties', 'primitive', 'ec

Fungsi di bawah ini digunakan untuk menghitung skor setiap kalimat berdasarkan frekuensi kata-kata yang muncul dalam teks, yang akan digunakan dalam proses ekstraksi kalimat teratas dalam pendekatan extractive text summarization.

In [ ]:
# Fungsi untuk menghitung skor setiap kalimat
def calculate_scores(sentences, freq_dist, stop_words):
    scores = {}
    for index, sentence in enumerate(sentences):
        score = 0
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word.isalnum() and word not in stop_words]
        for word in words:
            score += freq_dist[word]
        scores[index] = score
    return scores

sentences adalah daftar kalimat yang akan dihitung skornya.

freq_dist adalah frekuensi distribusi kata-kata yang telah dihitung sebelumnya.

stop_words adalah kumpulan stopwords (kata-kata umum yang diabaikan dalam pemrosesan teks).

Hasilnya adalah dictionary scores yang berisi skor untuk setiap kalimat dalam teks. Skor tersebut didasarkan pada jumlah frekuensi kemunculan kata-kata yang relevan dalam setiap kalimat. Skor ini akan digunakan untuk mengekstrak kalimat-kalimat teratas dalam proses ringkasan teks.

In [ ]:
# Menghitung skor setiap kalimat
scores = calculate_scores(sentences, freq_dist, stop_words)
print(scores)

{0: 73, 1: 76, 2: 101, 3: 65, 4: 75, 5: 98, 6: 72, 7: 10, 8: 72, 9: 114, 10: 116, 11: 77, 12: 85, 13: 64, 14: 56, 15: 133, 16: 117, 17: 68, 18: 97, 19: 15, 20: 12, 21: 158, 22: 64, 23: 37, 24: 57}


Fungsi ini berguna untuk memilih kalimat-kalimat dengan skor tertinggi yang akan digunakan sebagai ringkasan teks dalam pendekatan extractive text summarization (yaitu pendekatan dalam pembuatan ringkasan teks yang fokus pada pengambilan kalimat-kalimat penting dari teks asli untuk membentuk ringkasan yang mewakili informasi inti. Pendekatan ini tidak melibatkan proses generasi teks baru, melainkan mengandalkan ekstraksi kalimat yang sudah ada).

In [ ]:
# Fungsi untuk mengekstrak kalimat-kalimat teratas berdasarkan skor
def extract_top_sentences(sentences, scores, n):
    top_sentences = []
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for i in range(n):
        index = sorted_scores[i][0]
        top_sentences.append(sentences[index])
    return top_sentences

Hasilnya adalah daftar top_sentences yang berisi kalimat-kalimat teratas berdasarkan skor. Dalam kasus ini, kita mengambil 3 kalimat dengan skor tertinggi dari teks.

In [ ]:
# Mengambil top 3 kalimat berdasarkan skor
top_sentences = extract_top_sentences(sentences, scores, 3)
top_sentences

['The purpose of this review is to outline the salient biological properties of stem cell populations, highlight the main bioengineering challenges in the development of stem cell–based technologies, and focus on how recent information about parameters that may influence stem cell self-renewal and differentiation impact the design of these bioprocesses.',
 'Optimization of culture conditions for stem cell growth and differentiation has been further hindered by the need for retrospective assays of cell function to quantify stem cell responses to candidate culture parameters.',
 'Although these assays have been, and continue to be, exceedingly useful for describing properties of stem cells, their often lengthy and complicated nature represents a significant bottleneck in stem cell bioprocess development.']

In [ ]:
# Menampilkan ringkasan teks
summary = ' '.join(top_sentences)
print("Ringkasan Teks:")
print(summary)


Ringkasan Teks:
The purpose of this review is to outline the salient biological properties of stem cell populations, highlight the main bioengineering challenges in the development of stem cell–based technologies, and focus on how recent information about parameters that may influence stem cell self-renewal and differentiation impact the design of these bioprocesses. Optimization of culture conditions for stem cell growth and differentiation has been further hindered by the need for retrospective assays of cell function to quantify stem cell responses to candidate culture parameters. Although these assays have been, and continue to be, exceedingly useful for describing properties of stem cells, their often lengthy and complicated nature represents a significant bottleneck in stem cell bioprocess development.


Output di atas Membentuk ringkasan teks dengan menggabungkan kalimat-kalimat 3 teratas menggunakan function join.